# Vitessce Data Preparation Tutorial

# Export data to local files

## 1. Import dependencies

We need to import the classes and functions that we will be using from the corresponding packages.

In [ ]:
import os
import json
from urllib.parse import quote_plus
from os.path import join, isfile, isdir
from urllib.request import urlretrieve
from anndata import read_h5ad
import scanpy as sc

from vitessce import (
    VitessceWidget,
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    AnnDataWrapper,
)
from vitessce.data_utils import (
    optimize_adata,
    VAR_CHUNK_SIZE,
)

## 2. Download and process data

For this example, we need to download a dataset from the COVID-19 Cell Atlas https://www.covid19cellatlas.org/index.healthy.html#habib17.

In [ ]:
adata_filepath = join("data", "habib17.processed.h5ad")
if not isfile(adata_filepath):
    os.makedirs("data", exist_ok=True)
    urlretrieve('https://covid19.cog.sanger.ac.uk/habib17.processed.h5ad', adata_filepath)

adata = read_h5ad(adata_filepath)
top_dispersion = adata.var["dispersions_norm"][
    sorted(
        range(len(adata.var["dispersions_norm"])),
        key=lambda k: adata.var["dispersions_norm"][k],
    )[-51:][0]
]
adata.var["top_highly_variable"] = (
    adata.var["dispersions_norm"] > top_dispersion
)

In [ ]:
zarr_filepath = join("data", "habib17.processed.zarr")
if not isdir(zarr_filepath):
    adata = optimize_adata(
        adata,
        obs_cols=["CellType"],
        obsm_keys=["X_umap"],
        var_cols=["top_highly_variable"],
        optimize_X=True,
    )
    adata.write_zarr(zarr_filepath, chunks=[adata.shape[0], VAR_CHUNK_SIZE])

## 3. Create the Vitessce configuration

Set up the configuration by adding the views and datasets of interest.

In [ ]:
vc = VitessceConfig(schema_version="1.0.15", name='Habib et al', description='COVID-19 Healthy Donor Brain')
dataset = vc.add_dataset(name='Brain').add_object(AnnDataWrapper(
        base_path=zarr_filepath,
        obs_embedding_elems=["obsm/X_umap"],
        obs_embedding_names=["UMAP"],
        obs_set_elems=["obs/CellType"],
        obs_set_names=["Cell Type"],
        obs_feature_matrix_elem="X",
        feature_filter_elem="var/top_highly_variable"
))
scatterplot = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="X_umap")
cell_sets = vc.add_view(cm.OBS_SETS, dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
heatmap = vc.add_view(cm.HEATMAP, dataset=dataset)
vc.layout((scatterplot | (cell_sets / genes)) / heatmap);

## 4. Export files to a local directory

The `.export(to='files')` method on the view config instance will export files to the specified directory `out_dir`. The `base_url` parameter is required so that the file URLs in the view config point to the location where you ultimately intend to serve the files.

In [ ]:
config_dict = vc.export(to='files', base_url='http://localhost:3000', out_dir='./test')

## 5. Serve the files

Now that the files have been saved to the `./test` directory, they can be served by any static web server.

If you would like to serve the files locally, we recommend [http-server](https://github.com/http-party/http-server) which can be installed with NPM or Homebrew:
```sh
cd test
http-server ./ --cors -p 3000
```

## 6. View on vitessce.io

The returned view config dict can be converted to a URL, and if the files are served on the internet (rather than locally), this URL can be used to share the interactive visualizations with colleagues.

In [ ]:
vitessce_url = "http://vitessce.io/?url=data:," + quote_plus(json.dumps(config_dict))
import webbrowser
webbrowser.open(vitessce_url)